In [10]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

## 
Assumptions:
1. Number of people served in food pantry data = number of pounds
2. Approx Capacity = "Big M"<<Total_Supply to start
3. 

In [16]:
CENSUS_TRACTS_GEOJSON = "raw/census_tracts.geojson"   # e.g., 2020 Census tracts GeoJSON/GeoPackage/SHAPE
METRICS    = "data/tabular/nta_with_metrics.csv"                    # e.g., "data/supply_gap_by_neighborhood.csv" or None
PANTRIES_GEOJSON = "data/geo/pantries.geojson"
PANTRIES_PATH = "data/tabular/pantries.csv" 

### Information We Will Calculate to use in our optimization model:
1. Number of pantries within each NTA (neighborhood tabulation area) - coincides with metrics of food supply gap
2. Pairwise distance between all pantries and centroids of NTAs

In [19]:
# A: Food pantries
pantries = pd.read_csv(PANTRIES_PATH)

NUM_PANTRIES = pantries.shape[0]
print(f"Number of food pantries: {NUM_PANTRIES}")

pantries.head()

Number of food pantries: 515


,program,lat,lng,type_fp,type_sk,phone,address,boro,zip,loc_info,status,program_type,fp_hours,sk_hours
0,CHURCH OF ST. NICHOLAS OF TOLENTINE,40.862216,-73.905673,FP,NaN,(718) 295-6800,"2345 University Ave, Bronx, New York, 10468",BX,10468,BASEMENT FORDHAM RD ENTRANCE,Open,FP,Tue 09:00 AM-11:00 AM,NaN
1,BREAD OF LIFE FOOD PANTRY,40.826425,-73.878539,FP,NaN,(347) 235-3723,"1104 Elder Ave, Bronx, New York, 10472",BX,10472,#15,Open,FP,Sat 01:30 PM-04:30 PM,NaN
2,CHANCE FOR CHILDREN,40.665694,-73.958475,FP,NaN,(347) 616-3228,"11 Mc Keever Pl, Brooklyn, New York, 11225",BK,11225,1ST FLOOR (BASEMENT LEVEL),Open,FP,Fri 10:00 AM-12:30 PM,NaN
3,HEALTH ESSENTIAL ASSOCIATION INC (BK),40.598607,-73.955282,FP,NaN,(646) 515-6898,"2101 E 16th St, Brooklyn, New York, 11229",BK,11229,2ND FLOOR,Open,FP,Fri 10:00 AM-12:00 PM,NaN
4,AIDS CENTER OF QUEENS COUNTY (WOODSIDE),40.744659,-73.902187,FPHA,NaN,(718) 472-9400,"62-07 Woodside Ave, Woodside, New York, 11377",QN,11377,3RD FLOOR,Open,FP,Tue 08:00 AM-10:00 AM,NaN


**Pantries Per Neighborhood**

In [20]:
metrics_by_nta = pd.read_csv(METRICS)
NUM_NEIGHBORHOODS = metrics_by_nta.shape[0]
print(f"Number of neighborhoods: {NUM_NEIGHBORHOODS}")

metrics_by_nta.head()

Number of neighborhoods: 197


,nta2020,Year,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank,pantry_count
0,BK0101,2025-01-01,Greenpoint,"1,153,881.91747087",15.74%,741.51%,0.36,6.811714,44.0,1
1,BK0102,2025-01-01,Williamsburg,"1,011,421.07761282",16.48%,694.02%,0.38,6.326930,75.0,1
2,BK0103,2025-01-01,South Williamsburg,"2,090,950.72621075",27.58%,967.98%,0.60,8.033649,5.0,1
3,BK0104,2025-01-01,East Williamsburg,"1,224,484.03196005",21.34%,739.65%,0.42,6.832984,43.0,1
4,BK0201,2025-01-01,Brooklyn Heights,"322,651.410235809",10.05%,407.41%,0.44,5.637125,139.0,0


In [22]:
# B: Supply gap by neighborhood
gap_per_nta= metrics_by_nta[['nta2020','Supply Gap (lbs.)']]


In [23]:
# C: Pantries per NTA:
pantries_per_nta = metrics_by_nta[['nta2020','pantry_count']]

In [28]:
# If PANTRIES_PATH is CSV, specify columns:
PANTRY_LAT_COL = "lat"
PANTRY_LON_COL = "lng"

CRS_LATLON    = "EPSG:4326"
CRS_PROJECTED = "EPSG:6539"  # NYC-appropriate projected CRS

In [30]:
for col in [PANTRY_LAT_COL, PANTRY_LON_COL]:
    assert col in pantries.columns, f"Missing column in pantry CSV: {col}"
geo_pantries = pantries.dropna(subset=[PANTRY_LAT_COL, PANTRY_LON_COL])
geometry = gpd.points_from_xy(geo_pantries[PANTRY_LON_COL], geo_pantries[PANTRY_LAT_COL], crs=CRS_LATLON)
pantries = gpd.GeoDataFrame(geo_pantries, geometry=geometry)

pantries_ll = pantries.to_crs(CRS_LATLON)
pantries_proj = pantries_ll.to_crs(CRS_PROJECTED)

print("Pantries:", pantries_ll.shape)
pantries_ll.head(2)

Pantries: (515, 15)


,program,lat,lng,type_fp,type_sk,phone,address,boro,zip,loc_info,status,program_type,fp_hours,sk_hours,geometry
0,CHURCH OF ST. NICHOLAS OF TOLENTINE,40.862216,-73.905673,FP,NaN,(718) 295-6800,"2345 University Ave, Bronx, New York, 10468",BX,10468,BASEMENT FORDHAM RD ENTRANCE,Open,FP,Tue 09:00 AM-11:00 AM,NaN,POINT (-73.90567 40.86222)
1,BREAD OF LIFE FOOD PANTRY,40.826425,-73.878539,FP,NaN,(347) 235-3723,"1104 Elder Ave, Bronx, New York, 10472",BX,10472,#15,Open,FP,Sat 01:30 PM-04:30 PM,NaN,POINT (-73.87854 40.82642)
